# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
import json
with open("data/contributors_sample.json", 'r') as f:
    k = json.load(f)
    print(k[:3])
    f.close()
k

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
mails = {}
for i in k:
    mails.update({i["mail"][i["mail"].find("@") + 1:]: 0})
mails.keys()

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def find_man(username):
    for i in k:
        if i["username"] == username:
            return i["name"]
    raise ValueError
find_man("uhebert")

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
man, woman = 0, 0
for i in k:
    if i["sex"] == "M":
        man += 1
    elif i["sex"] == "F":
        woman += 1
print(f"man {man}\nwoman {woman}")

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
import pandas as pd

di = {}

for i in ["id", "username", "sex"]:
    di.update(dict.fromkeys([i], [j[i] for j in k]))
    
contributors = pd.DataFrame(di)

contributors

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
recipes = pd.read_csv('../02_pandas/recipes_sample.csv')
recipes.merge(contributors, how='inner', on=None, left_on="contributor_id", right_on="id", left_index=False, right_index=True, sort=False, suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
import pickle

jobs = {}

for i in k:
    for j in i["jobs"]:
        try:
            jobs[j].append(i["username"])
        except KeyError:
            t = [i["username"]]
            jobs.update({j: t})
            
jobs

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
with open("data/job_people.pickle", "wb") as f:
    pickle.dump(jobs, f)

with open("data/job_people.json", "w") as f:
    json.dump(jobs, f)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open("data/job_people.pickle", "rb") as f:
    print(pickle.load(f))

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
import xml.etree.ElementTree as ET

recipes123 = {}

tree = ET.parse('data/steps_sample.xml')

for i in tree.getroot():
    steps = []
    for j in i[1]:
        steps.append(j.text)
    recipes123.update({i[0].text: steps})
    
with open("data/steps_sample.json", "w") as f:
    json.dump(recipes123, f)

recipes123

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
ids1_steps = {}

for i in recipes123:
    try:
        ids1_steps[str(len(recipes123[i]))].append(i)
    except KeyError:
        ids1_steps.update({str(len(recipes123[i])): [i]})
        
ids1_steps

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
recipes_with_time = {}

for i in tree.getroot():
    
    no_time_attr = 0
    
    for j in i[1]:
        
        try:
            tmp = []
            for i1 in i[1]:
                tmp.append(i1.text)
            recipes_with_time.update({i[0].text:  tmp})
            break
        except KeyError:
            no_time_attr += 1

        
recipes_with_time

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
for index, row in recipes.iterrows():
    
    if pd.isnull(row["n_steps"]):
        recipes.at[index, "n_steps"] = float(len(recipes123[str(row["id"])]))
        
for index, row in recipes.iterrows():
    print(row["n_steps"])

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
def check():
    for index, row in recipes.iterrows():
        if pd.isnull(row["n_steps"]):
            return False
    return True


if check():
    recipes["n_steps"] = recipes["n_steps"].apply("int64") 

    for index, row in recipes.iterrows():
        print(row["n_steps"])

recipes.to_csv("data/recipes_sample_with_filled_nsteps.csv")

In [ ]:
ls data